In [1]:
import pandas as pd
import numpy as np
import re
import os

import nltk
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
import os

import pickle

In [23]:
tweets = pd.read_csv("_persistentId_persistentId")
print(tweets.shape)
tweets.head(10)

(52542, 10)


,author,content,country,date_time,id,language,latitude,longitude,number_of_likes,number_of_shares
0,katyperry,Is history repeating itself...?#DONTNORMALIZEH...,NaN,12/01/2017 19:52,8.196330e+17,en,NaN,NaN,7900,3472
1,katyperry,@barackobama Thank you for your incredible gra...,NaN,11/01/2017 08:38,8.191010e+17,en,NaN,NaN,3689,1380
2,katyperry,Life goals. https://t.co/XIn1qKMKQl,NaN,11/01/2017 02:52,8.190140e+17,en,NaN,NaN,10341,2387
3,katyperry,Me right now 🙏🏻 https://t.co/gW55C1wrwd,NaN,11/01/2017 02:44,8.190120e+17,en,NaN,NaN,10774,2458
4,katyperry,SISTERS ARE DOIN' IT FOR THEMSELVES! 🙌🏻💪🏻❤️ ht...,NaN,10/01/2017 05:22,8.186890e+17,en,NaN,NaN,17620,4655
5,katyperry,happy 96th gma #fourmoreyears! 🎈 @ LACMA Los A...,NaN,09/01/2017 01:00,8.182610e+17,en,NaN,NaN,8242,2164
6,katyperry,"Kyoto, Japan \r\n1. 5. 17. https://t.co/o28M0v...",NaN,06/01/2017 03:57,8.172180e+17,tl,NaN,NaN,7171,1910
7,katyperry,🇯🇵 @ Sanrio Puroland https://t.co/eXVev5UMBx,NaN,02/01/2017 09:31,8.158530e+17,cy,NaN,NaN,7766,2468
8,katyperry,2017 resolution: to embody authenticity!,NaN,01/01/2017 03:31,8.154000e+17,en,NaN,NaN,23662,8430
9,katyperry,sisters. https://t.co/5ZE21x2aNk,NaN,27/12/2016 04:04,8.135960e+17,en,NaN,NaN,9324,2637


In [24]:
apostrophe_dict = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

short_word_dict = {
"121": "one to one",
"a/s/l": "age, sex, location",
"adn": "any day now",
"afaik": "as far as I know",
"afk": "away from keyboard",
"aight": "alright",
"alol": "actually laughing out loud",
"b4": "before",
"b4n": "bye for now",
"bak": "back at the keyboard",
"bf": "boyfriend",
"bff": "best friends forever",
"bfn": "bye for now",
"bg": "big grin",
"bta": "but then again",
"btw": "by the way",
"cid": "crying in disgrace",
"cnp": "continued in my next post",
"cp": "chat post",
"cu": "see you",
"cul": "see you later",
"cul8r": "see you later",
"cya": "bye",
"cyo": "see you online",
"dbau": "doing business as usual",
"fud": "fear, uncertainty, and doubt",
"fwiw": "for what it's worth",
"fyi": "for your information",
"g": "grin",
"g2g": "got to go",
"ga": "go ahead",
"gal": "get a life",
"gf": "girlfriend",
"gfn": "gone for now",
"gmbo": "giggling my butt off",
"gmta": "great minds think alike",
"h8": "hate",
"hagn": "have a good night",
"hdop": "help delete online predators",
"hhis": "hanging head in shame",
"iac": "in any case",
"ianal": "I am not a lawyer",
"ic": "I see",
"idk": "I don't know",
"imao": "in my arrogant opinion",
"imnsho": "in my not so humble opinion",
"imo": "in my opinion",
"iow": "in other words",
"ipn": "I’m posting naked",
"irl": "in real life",
"jk": "just kidding",
"l8r": "later",
"ld": "later, dude",
"ldr": "long distance relationship",
"llta": "lots and lots of thunderous applause",
"lmao": "laugh my ass off",
"lmirl": "let's meet in real life",
"lol": "laugh out loud",
"ltr": "longterm relationship",
"lulab": "love you like a brother",
"lulas": "love you like a sister",
"luv": "love",
"m/f": "male or female",
"m8": "mate",
"milf": "mother I would like to fuck",
"oll": "online love",
"omg": "oh my god",
"otoh": "on the other hand",
"pir": "parent in room",
"ppl": "people",
"r": "are",
"rofl": "roll on the floor laughing",
"rpg": "role playing games",
"ru": "are you",
"shid": "slaps head in disgust",
"somy": "sick of me yet",
"sot": "short of time",
"thanx": "thanks",
"thx": "thanks",
"ttyl": "talk to you later",
"u": "you",
"ur": "you are",
"uw": "you’re welcome",
"wb": "welcome back",
"wfm": "works for me",
"wibni": "wouldn't it be nice if",
"wtf": "what the fuck",
"wtg": "way to go",
"wtgp": "want to go private",
"ym": "young man",
"gr8": "great"
}

emoticon_dict = {
":)": "happy",
":‑)": "happy",
":-]": "happy",
":-3": "happy",
":->": "happy",
"8-)": "happy",
":-}": "happy",
":o)": "happy",
":c)": "happy",
":^)": "happy",
"=]": "happy",
"=)": "happy",
"<3": "happy",
":-(": "sad",
":(": "sad",
":c": "sad",
":<": "sad",
":[": "sad",
">:[": "sad",
":{": "sad",
">:(": "sad",
":-c": "sad",
":-< ": "sad",
":-[": "sad",
":-||": "sad"
}

fail_dict = {
    ":)": ":\)",
    ":‑)": ":‑\)",
    ":-]": ":-\]",
    "8-)": "8-\)",
    ":o)": ":o\)",
    ":c)": ":c\)",
    ":^)": ":\^\)",
    "=)": "=\)",
    ":-(": ":-\(",
    ":(": ":\(",
    ":[": ":\[",
    ">:[": ">:\[",
    ">:(": ">:\(",
    ":-[": ":-\[",
    ":-||": ":-\|\|"
}

In [25]:
def preprocess_txt(line):
    spls = re.sub(r"https?://[^,\s]+,?", " ", line)
    spls = re.sub(r'\d+\s?', " ", spls).strip()
    spls = re.sub(r'(\b\w+)\s+\1', r'\1', spls)
    spls = re.sub(r'[^а-яА-Яё ]', ' ', spls)
    spls = " ".join(re.sub(html_chars, ' ', spls).split())
    spls = "".join(i if i not in exclude else ' ' for i in spls.strip()).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

def clean_name(name, text):
    return str(re.sub(name, '', text))


def change_of_dict(dictionary, text):
    for el in text.split():
        if el in dictionary:
            if el in fail_dict:
                el_temp = fail_dict[el]
                text = re.sub(el_temp, dictionary[el], text)
            else:
                text = re.sub(el, dictionary[el], text)
    return text


def clean_text(text):
    '''
    очистка текста

    на выходе очищеный текст
    '''
    if not isinstance(text, str):
        text = str(text)

    vect_func = np.vectorize(clean_name)
    
    text = html.unescape(text)              # 1. Заменим html-сущности
    list_name = re.findall("@[\w]*", text)  # 2. Удалим @user из всех твитов

    if list_name:
        text = vect_func(list_name, text)
        text = str(text[0])

    text = text.lower()                     # 3. Изменим регистр твитов на нижний с помощью .lower().
    
    text = re.sub(r"https?://[^,\s]+,?", " ", text)

    # 4. Заменим сокращения с апострофами на значение полную версию слова
    text = change_of_dict(apostrophe_dict, text)

    # 5. Заменим сокращения на их полные формы
    text = change_of_dict(short_word_dict, text)

    # 6. Заменим эмотиконы
    text = change_of_dict(emoticon_dict, text)

    # 7. Заменим пунктуацию на пробелы
    text = re.sub(r'[^\w\s]', ' ', text)

    # 8. Заменим спец. символы на пробелы
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)

    # 9. Заменим числа на пробелы
    text = re.sub(r'[^a-zA-Z]', ' ', text)

    # 10. Удалим из текста слова длиной в 1 символ
    text = ' '.join([w for w in text.split() if len(w) > 1])

    return text

In [26]:
%%time
import html
from tqdm import tqdm
tqdm.pandas()

CPU times: total: 15.6 ms
Wall time: 1e+03 µs


In [27]:
# Запускаем очистку текста.
df_tweets = pd.DataFrame(tweets['content'].progress_apply(lambda x: clean_text(x)), columns=['content'])
df_tweets.columns = ['tweet']
df_tweets.head()

100%|██████████████████████████████████████████████████████████████████████████| 52542/52542 [00:07<00:00, 6711.20it/s]


,tweet
0,is history repeating itself dontnormalizehate
1,thank you for your incredible grace in leaders...
2,life goals
3,me right now
4,sisters are doin it for themselves


In [28]:
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk import tokenize as tknz
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Вадим\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [29]:
df_tweets['tweet_token'] = df_tweets['tweet'].progress_apply(lambda x: tknz.word_tokenize(x))
df_tweets.tweet_token

100%|██████████████████████████████████████████████████████████████████████████| 52542/52542 [00:12<00:00, 4324.80it/s]


0        [is, history, repeating, itself, dontnormalize...
1        [thank, you, for, your, incredible, grace, in,...
2                                            [life, goals]
3                                         [me, right, now]
4                [sisters, are, doin, it, for, themselves]
                               ...                        
52537           [life, could, not, be, better, right, now]
52538    [first, monday, back, in, action, had, would, ...
52539    [crime, shows, buddy, snuggles, the, perfect, ...
52540                                                   []
52541                                                   []
Name: tweet_token, Length: 52542, dtype: object

In [30]:
from nltk.corpus import stopwords
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [31]:
stop_words = set(stopwords.words("english"))

In [32]:
def del_stop_words(words, stop_words):
    return [word for word in words if word not in stop_words]

In [33]:
df_tweets['tweet_token_filtered'] = df_tweets['tweet_token'].progress_apply(lambda x: del_stop_words(x, stop_words))

100%|████████████████████████████████████████████████████████████████████████| 52542/52542 [00:00<00:00, 199781.09it/s]


In [34]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet

In [35]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

In [36]:
def used_stemmer(stemmer, words):
    return [stemmer.stem(word) for word in words]

In [37]:
df_tweets['tweet_stemmed'] = df_tweets['tweet_token_filtered'].progress_apply(lambda x: used_stemmer(stemmer, x))

100%|██████████████████████████████████████████████████████████████████████████| 52542/52542 [00:14<00:00, 3590.65it/s]


In [38]:
df_tweets.tweet_stemmed

0                        [histori, repeat, dontnormalizeh]
1               [thank, incred, grace, leadership, except]
2                                             [life, goal]
3                                                  [right]
4                                           [sister, doin]
                               ...                        
52537                         [life, could, better, right]
52538    [first, monday, back, action, would, say, mile...
52539    [crime, show, buddi, snuggl, perfect, sunday, ...
52540                                                   []
52541                                                   []
Name: tweet_stemmed, Length: 52542, dtype: object

In [39]:
def used_lemmatizer(lemmatizer, words):
    return [lemmatizer.lemmatize(word) for word in words]

In [40]:
df_tweets['tweet_lemmatized'] = df_tweets['tweet_token_filtered'].progress_apply(lambda x: used_lemmatizer(lemmatizer, x))

100%|██████████████████████████████████████████████████████████████████████████| 52542/52542 [00:06<00:00, 8069.72it/s]


In [42]:
df_tweets.tweet_lemmatized

0                  [history, repeating, dontnormalizehate]
1        [thank, incredible, grace, leadership, excepti...
2                                             [life, goal]
3                                                  [right]
4                                           [sister, doin]
                               ...                        
52537                         [life, could, better, right]
52538    [first, monday, back, action, would, say, mile...
52539    [crime, show, buddy, snuggle, perfect, sunday,...
52540                                                   []
52541                                                   []
Name: tweet_lemmatized, Length: 52542, dtype: object

In [43]:
df_tweets['tweet_stemmed'] = df_tweets['tweet_stemmed'].apply(lambda x: ' '.join(x))
df_tweets['tweet_lemmatized'] = df_tweets['tweet_lemmatized'].apply(lambda x: ' '.join(x))

In [44]:
f = open(r'df_dop_tweet.txt', 'wb')
pickle.dump(df_tweets, f)
f.close()

In [45]:
f = open(r'df_dop_tweet.txt', 'rb')
df_tweets_load = pickle.load(f)
f.close()
df_tweets_load.head()

,tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,is history repeating itself dontnormalizehate,"[is, history, repeating, itself, dontnormalize...","[history, repeating, dontnormalizehate]",histori repeat dontnormalizeh,history repeating dontnormalizehate
1,thank you for your incredible grace in leaders...,"[thank, you, for, your, incredible, grace, in,...","[thank, incredible, grace, leadership, excepti...",thank incred grace leadership except,thank incredible grace leadership exceptional
2,life goals,"[life, goals]","[life, goals]",life goal,life goal
3,me right now,"[me, right, now]",[right],right,right
4,sisters are doin it for themselves,"[sisters, are, doin, it, for, themselves]","[sisters, doin]",sister doin,sister doin
